<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GTFS" data-toc-modified-id="GTFS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GTFS</a></span></li><li><span><a href="#Attempt-to-join-based-on-minimum-distance-between-coordinates-(Haversine)" data-toc-modified-id="Attempt-to-join-based-on-minimum-distance-between-coordinates-(Haversine)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Attempt to join based on minimum distance between coordinates (Haversine)</a></span><ul class="toc-item"><li><span><a href="#Test-if-latter-4-digits-of-GTFS-==-DB-code" data-toc-modified-id="Test-if-latter-4-digits-of-GTFS-==-DB-code-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Test if latter 4 digits of GTFS == DB code</a></span></li><li><span><a href="#Export-dictionary" data-toc-modified-id="Export-dictionary-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Export dictionary</a></span></li></ul></li><li><span><a href="#SEPARATE-WORKFLOW:-GENERATING-STOP-TO-LAT/LON-DICT" data-toc-modified-id="SEPARATE-WORKFLOW:-GENERATING-STOP-TO-LAT/LON-DICT-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>SEPARATE WORKFLOW: GENERATING STOP TO LAT/LON DICT</a></span></li></ul></div>

In [2]:
import os
import json
import re
from math import radians, sin, cos, asin, sqrt

In [2]:
from IPython.core.display import display, HTML
display(HTML('<style>.container {width:90% !important;}</style>'))

In [3]:
os.chdir('/Users/davidodwyer/Documents/studyCS/Semester_3/data/original_raw_data')

db_json = open('busstopinformation.json')
db_json_str = db_json.read()
db_data = json.loads(db_json_str)

In [4]:
db_data

{'errorcode': '0',
 'errormessage': '',
 'numberofresults': 10085,
 'timestamp': '12/07/2019 15:21:31',
 'results': [{'stopid': '2',
   'displaystopid': '2',
   'shortname': 'Parnell Square',
   'shortnamelocalized': 'Cearnóg Parnell',
   'fullname': 'Parnell Square',
   'fullnamelocalized': '',
   'latitude': '53.35224111',
   'longitude': '-6.263695',
   'lastupdated': '08/07/2019 09:47:25',
   'operators': [{'name': 'bac',
     'routes': ['38', '38A', '38B', '38D', '46A', '46E']}]},
  {'stopid': '3',
   'displaystopid': '3',
   'shortname': 'Parnell Square',
   'shortnamelocalized': 'Cearnóg Parnell',
   'fullname': 'Parnell Square',
   'fullnamelocalized': '',
   'latitude': '53.35230694',
   'longitude': '-6.263783056',
   'lastupdated': '01/07/2019 09:29:13',
   'operators': [{'name': 'bac', 'routes': ['120', '122']}]},
  {'stopid': '4',
   'displaystopid': '4',
   'shortname': 'Parnell Square',
   'shortnamelocalized': 'Cearnóg Parnell',
   'fullname': 'Parnell Square',
   'full

In [5]:
db_data['results']

[{'stopid': '2',
  'displaystopid': '2',
  'shortname': 'Parnell Square',
  'shortnamelocalized': 'Cearnóg Parnell',
  'fullname': 'Parnell Square',
  'fullnamelocalized': '',
  'latitude': '53.35224111',
  'longitude': '-6.263695',
  'lastupdated': '08/07/2019 09:47:25',
  'operators': [{'name': 'bac',
    'routes': ['38', '38A', '38B', '38D', '46A', '46E']}]},
 {'stopid': '3',
  'displaystopid': '3',
  'shortname': 'Parnell Square',
  'shortnamelocalized': 'Cearnóg Parnell',
  'fullname': 'Parnell Square',
  'fullnamelocalized': '',
  'latitude': '53.35230694',
  'longitude': '-6.263783056',
  'lastupdated': '01/07/2019 09:29:13',
  'operators': [{'name': 'bac', 'routes': ['120', '122']}]},
 {'stopid': '4',
  'displaystopid': '4',
  'shortname': 'Parnell Square',
  'shortnamelocalized': 'Cearnóg Parnell',
  'fullname': 'Parnell Square',
  'fullnamelocalized': '',
  'latitude': '53.35256694',
  'longitude': '-6.264166111',
  'lastupdated': '01/07/2019 09:29:13',
  'operators': [{'name

In [6]:
number_stops = 0
for entry in db_data['results']:
    number_stops += 1
#     print(entry['stopid'])
print(number_stops)

10085


In [7]:
db_data['results'][0]['operators'][0]['name']

'bac'

In [8]:
# find all operators in dataset

operators = []

for entry in db_data['results']:
    operators.append(entry['operators'][0]['name'])
    
operators = set(operators)
operators = list(operators)
print(operators)
    

['ir', 'GAD', 'bac', 'LUAS', 'BE']


We infer from the following links that 'bac' specifies Dublin Bus serives:
    
https://data.smartdublin.ie/cgi-bin/rtpi/realtimebusinformation?stopid=184&format=json
    
https://data.gov.ie/dataset/real-time-passenger-information-rtpi-for-dublin-bus-bus-eireann-luas-and-irish-rail/resource/f56216c5-8366-4637-9fc7-8fcc11498294?inner_span=True
    
https://data.gov.ie/dataset/real-time-passenger-information-rtpi-for-dublin-bus-bus-eireann-luas-and-irish-rail/resource/c821fac3-ed2b-4901-b9ae-0dd1c6a748bf

In [9]:
# find the number of dublin bus stops

number_stops = 0
for entry in db_data['results']:
    if entry['operators'][0]['name'] == 'bac':
        number_stops += 1
#     print(entry['stopid'])
print(number_stops)

4245


In [17]:
db_stop_data = {}

In [18]:
db_data['results'][0]

{'stopid': '2',
 'displaystopid': '2',
 'shortname': 'Parnell Square',
 'shortnamelocalized': 'Cearnóg Parnell',
 'fullname': 'Parnell Square',
 'fullnamelocalized': '',
 'latitude': '53.35224111',
 'longitude': '-6.263695',
 'lastupdated': '08/07/2019 09:47:25',
 'operators': [{'name': 'bac',
   'routes': ['38', '38A', '38B', '38D', '46A', '46E']}]}

Checking slicing

In [19]:
db_data['results'][0]['stopid'][0]

'2'

In [20]:
db_data['results'][0]['latitude']

'53.35224111'

In [21]:
db_data['results'][0]['longitude']

'-6.263695'

In [22]:
db_data['results'][0]['shortname']

'Parnell Square'

In [23]:
# write a dictionary: latitude, longitude, 
# and name for each stop

for entry in db_data['results']:
    if entry['operators'][0]['name'] == 'bac':
        db_stop_data[entry['stopid']] = [float(entry['latitude']), float(entry['longitude']), entry['shortname']]

In [24]:
db_stop_data

{'2': [53.35224111, -6.263695, 'Parnell Square'],
 '3': [53.35230694, -6.263783056, 'Parnell Square'],
 '4': [53.35256694, -6.264166111, 'Parnell Square'],
 '6': [53.35274389, -6.264443056, 'Parnell Square'],
 '7': [53.35283611, -6.264561944, 'Parnell Square'],
 '8': [53.35328111, -6.265155, 'Parnell Square'],
 '10': [53.35338694, -6.265383889, 'Parnell Square'],
 '11': [53.35709389, -6.264366944, 'Dorset St'],
 '12': [53.35678889, -6.264623056, 'Dorset St'],
 '14': [53.35853694, -6.262723889, 'Dorset St'],
 '15': [53.36025111, -6.260973056, 'Dorset St'],
 '16': [53.36641889, -6.305635, 'Nephin Road'],
 '17': [53.36306694, -6.258398056, 'Drumcondra Rd'],
 '18': [53.36584889, -6.255931111, 'Drumcondra Rd'],
 '19': [53.36713806, -6.255493056, 'Drumcondra Rd'],
 '21': [53.37009194, -6.254291111, 'Drumcondra Rd'],
 '22': [53.37350306, -6.252781944, 'Home Farm Road'],
 '23': [53.37423194, -6.256245, 'Home Farm Road'],
 '24': [53.375135, -6.260346111, 'Home Farm Road'],
 '25': [53.37590306, 

In [25]:
len(db_stop_data)

4245

# GTFS

In [10]:
os.chdir('/Users/davidodwyer/Documents/studyCS/Semester_3/data/derived_data')

gtfs_json = open('GTFS_stops_geo2.json')
gtfs_json_str = gtfs_json.read()
gtfs_data = json.loads(gtfs_json_str)

In [11]:
gtfs_data

{'8220DB004495': [53.346186780100005,
  -6.2569701147800005,
  "Dublin City South, Hawkin's Street"],
 '8220DB001279': [53.3442604936, -6.261704666490001, 'College Green'],
 '8220DB001934': [53.3442905616,
  -6.2656234691999995,
  'Temple Bar, Dame Street'],
 '8220DB002310': [53.3424457018,
  -6.269798428370001,
  'Dublin City South, Werburgh Street'],
 '8220DB002311': [53.33828528270001,
  -6.2704924670399995,
  'Dublin City South, Kevin Street Garda Station'],
 '8220DB005025': [53.339763015699994,
  -6.27758105967,
  'Warrenmount, The Coombe'],
 '8220DB004854': [53.3381491415, -6.27910275885, 'Warrenmount, Ardee Street'],
 '8220DB001438': [53.3363829769,
  -6.279414136180001,
  'Warrenmount, Saint Thomas Road'],
 '8220DB004855': [53.33460176770001,
  -6.2793056488,
  "Warrenmount, O'Donovan Road"],
 '8220DB001439': [53.3340584157,
  -6.28276609921,
  'Warrenmount, Donore Credit Union'],
 '8220DB001440': [53.3319900594,
  -6.2833146521700005,
  'Warrenmount, Donore Avenue'],
 '8220DB0

In [12]:
len(gtfs_data)

4239

In [13]:
gtfs_data['8220DB004495']

[53.346186780100005, -6.2569701147800005, "Dublin City South, Hawkin's Street"]

# Attempt to join based on minimum distance between coordinates (Haversine)

In [14]:
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    return 2 * 6371 * asin(sqrt(a))

In [15]:
def map_stops_by_proximity(db_dict, gtfs_dict):
    
    # declare dict that will contain GTFS to DB stop mappings
    
    mapping = dict()
    
    # iterate through the DB stops
    
    for key, value in db_dict.items():
        db_lat = db_dict[key][0]
        db_lon = db_dict[key][1]
        
        # declare a dict for distances between DB stop and all GTFS stops
        # decalre a dict to hold the keys iterated through
        
        distances = []
        keys = []
        
        # iterate through the GTFS stops
        
        for k, v in gtfs_dict.items():
            gtfs_lat = gtfs_dict[k][0]
            gtfs_lon = gtfs_dict[k][1]
            
            #calcuate the distance between the two lat-lons
            
            distance = haversine(db_lon, db_lat, gtfs_lon, gtfs_lat)
            
            # add the distance in sequence
            # add the key in sequence
            distances.append(distance)
            keys.append(k)
            
        # find the index of the shortest distance
        
        index_smallest_distance = index_min = min(range(len(distances)), key=distances.__getitem__)
        
        # add to the master dictionary: DB stop = rightly indexed GTFS
        
        mapping[key] = keys[index_smallest_distance]
        
    return mapping


In [26]:
# mappiung according to Haversein

mapping = map_stops_by_proximity(db_stop_data, gtfs_data)

In [27]:
for k, v in mapping.items():
    print(k, v)

2 8220DB000002
3 8220DB000003
4 8220DB000004
6 8220DB000006
7 8220DB000007
8 8220DB000008
10 8220DB000010
11 8220DB000011
12 8220DB000012
14 8220DB000014
15 8220DB000015
16 8220DB000016
17 8220DB000017
18 8220DB000018
19 8220DB000019
21 8220DB000021
22 8220DB000022
23 8220DB000023
24 8220DB000024
25 8220DB000025
27 8220DB000027
28 8220DB000028
29 8220DB000029
30 8220DB000030
31 8220DB000031
32 8220DB000032
33 8220DB000033
35 8220DB000035
36 8220DB000036
37 8220DB000037
38 8220DB000038
39 8220DB000039
40 8220DB000040
41 8220DB000041
42 8220DB000042
43 8220DB000043
44 8220DB000044
45 8220DB000045
46 8220DB000046
47 8220DB000047
48 8220DB000048
49 8220DB000049
51 8220DB000051
52 8220DB000052
53 8220DB000053
54 8220DB000054
55 8220DB000055
56 8220DB000056
57 8220DB000057
58 8220DB000058
59 8220DB000059
60 8220DB000060
63 8220DB000063
64 8220DB000064
65 8220DB000065
66 8220DB000066
68 8220DB000068
69 8220DB000069
71 8220DB000071
72 8220DB000072
73 8220DB000073
74 8220DB000074
75 8220DB00007

847 8220DB000847
848 8220DB000848
849 8220DB000849
850 8220DB000850
851 8220DB000851
852 8220DB000852
853 8220DB000853
854 8220DB000854
855 8220DB000855
856 8220DB000856
857 8220DB000857
858 8220DB000858
859 8250DB000859
860 8250DB000860
861 8250DB000861
862 8250DB000862
875 8250DB000875
876 8250DB000876
877 8250DB000877
878 8250DB000878
879 8250DB000879
880 8250DB000880
881 8220DB000881
882 8220DB000882
883 8220DB000883
884 8220DB000884
885 8220DB000885
886 8220DB000886
887 8220DB000887
893 8220DB000893
894 8220DB000894
895 8220DB000895
896 8220DB000896
897 8220DB000897
903 8220DB000903
904 8220DB000904
905 8240DB000905
906 8220DB000906
907 8220DB000907
908 8220DB000908
909 8220DB000909
910 8240DB000910
911 8240DB000911
912 8240DB000912
913 8240DB000913
914 8240DB000914
915 8240DB000915
916 8240DB000916
917 8240DB000917
918 8240DB000918
919 8240DB000919
924 8240DB000924
925 8240DB000925
927 8240DB000927
928 8240DB000928
929 8240DB000929
930 8240DB000930
931 8240DB000931
938 8240DB0009

1575 8220DB001575
1576 8220DB001576
1577 8220DB001577
1578 8220DB001578
1579 8220DB001579
1580 8240DB001580
1581 8240DB001581
1582 8220DB001582
1584 8220DB001584
1585 8220DB001585
1586 8220DB001586
1587 8220DB001587
1591 8220DB001591
1592 8220DB001592
1593 8220DB001593
1594 8220DB001594
1595 8220DB001595
1596 8220DB001596
1597 8220DB001597
1598 8220DB001598
1599 8220DB001599
1600 8220DB001600
1604 8220DB001604
1606 8220DB001606
1607 8220DB001607
1608 8220DB001608
1609 8220DB001609
1610 8220DB001610
1614 8220DB001614
1615 8220DB001615
1616 8220DB001616
1617 8220DB001617
1618 8220DB001618
1619 8220DB001619
1620 8220DB001620
1622 8240DB001622
1623 8240DB001623
1624 8240DB001624
1625 8240DB001625
1626 8240DB001626
1627 8240DB001627
1628 8240DB001628
1629 8240DB001629
1630 8240DB001631
1631 8240DB001631
1632 8240DB001632
1633 8240DB001633
1634 8240DB001628
1635 8240DB001635
1636 8240DB001636
1637 8240DB001637
1638 8240DB001638
1639 8240DB001639
1640 8240DB001640
1641 8220DB001641
1642 8220D

2174 8230DB002174
2176 8230DB002176
2177 8230DB002177
2178 8230DB002178
2181 8220DB002181
2182 8220DB002182
2183 8220DB002183
2184 8220DB002184
2185 8220DB002185
2186 8220DB002186
2187 8220DB002187
2188 8220DB002188
2189 8220DB002189
2190 8220DB002190
2191 8220DB002191
2192 8220DB002192
2193 8220DB002193
2194 8220DB002194
2195 8220DB002195
2196 8220DB002196
2197 8220DB002197
2198 8220DB002198
2200 8220DB002200
2201 8230DB002201
2202 8230DB002202
2203 8230DB002203
2204 8230DB002204
2205 8230DB002205
2206 8230DB002206
2207 8230DB002207
2208 8230DB002208
2210 8230DB002210
2211 8230DB002211
2213 8230DB002213
2214 8230DB002214
2215 8230DB002215
2216 8230DB002216
2217 8230DB002217
2218 8230DB002218
2219 8230DB002219
2220 8230DB002220
2221 8230DB002221
2223 8230DB002223
2224 8230DB002224
2225 8230DB002225
2227 8230DB002227
2229 8230DB002229
2230 8230DB002230
2231 8230DB002231
2232 8230DB002232
2233 8230DB002233
2234 8230DB002234
2235 8230DB002235
2236 8230DB002215
2238 8230DB002238
2239 8230D

2910 8220DB002910
2911 8220DB002911
2912 8220DB002912
2914 8220DB002914
2915 8220DB002915
2916 8220DB002916
2917 8220DB002917
2918 8220DB002918
2919 8230DB002919
2920 8230DB002920
2921 8230DB002921
2922 8230DB002922
2923 8230DB002923
2924 8230DB002924
2925 8230DB002925
2927 8230DB002927
2929 8230DB002929
2930 8230DB002930
2933 8230DB002933
2934 8230DB002934
2935 8230DB002935
2936 8230DB002936
2937 8230DB002937
2938 8230DB002938
2940 8230DB002940
2941 8230DB002941
2942 8230DB002942
2943 8230DB002943
2944 8230DB002944
2945 8230DB002945
2946 8230DB002946
2947 8230DB002947
2948 8230DB002948
2949 8230DB002949
2950 8230DB002950
2954 8230DB002954
2955 8230DB002955
2956 8230DB002956
2957 8220DB002957
2958 8220DB002958
2959 8240DB002959
2961 8240DB002961
2962 8350DB002962
2963 8350DB002963
2964 8350DB002964
2965 8230DB002965
2966 8230DB002966
2967 8230DB002967
2968 8230DB002968
2969 8250DB002969
2970 8250DB002970
2971 8250DB002971
2975 8250DB002975
2976 8250DB002976
2977 8250DB002977
2978 8250D

3632 8240DB003632
3633 8240DB003633
3634 8240DB003634
3635 8240DB003635
3636 8240DB003636
3639 8240DB003639
3640 8240DB003640
3641 8240DB003641
3642 8240DB003642
3643 8240DB003643
3644 8240DB003644
3645 8240DB003645
3646 8240DB003646
3647 8240DB003647
3648 8240DB003648
3649 8240DB003649
3650 8240DB003650
3651 8240DB003651
3652 8240DB003652
3653 8240DB003653
3654 8240DB003654
3655 8240DB003655
3656 8240DB003656
3657 8240DB003657
3658 8240DB003658
3659 8240DB003659
3660 8240DB003660
3661 8240DB003661
3662 8240DB003662
3663 8240DB003663
3664 8240DB003664
3665 8240DB003665
3669 8240DB003669
3670 8240DB003670
3671 8240DB003671
3672 8240DB003672
3674 8240DB003674
3675 8240DB003675
3676 8240DB003676
3677 8240DB003677
3678 8240DB003678
3679 8240DB003679
3682 8240DB003682
3686 8240DB003686
3689 8240DB003689
3690 8240DB003690
3691 8240DB003691
3692 8240DB003692
3694 8240DB003694
3695 8240DB003695
3696 8240DB003696
3697 8240DB003697
3698 8240DB003698
3699 8240DB003699
3701 8240DB003701
3702 8240D

4550 8230DB004550
4551 8230DB004551
4552 8230DB004555
4554 8230DB004554
4555 8230DB004555
4557 8230DB004557
4558 8230DB004558
4559 8230DB004559
4560 8230DB004560
4561 8230DB004561
4562 8230DB004562
4563 8220DB004563
4564 8230DB004564
4565 8250DB004565
4566 8250DB004566
4567 8250DB004567
4568 8250DB004568
4569 8250DB004569
4570 8250DB004570
4571 8250DB004571
4573 8240DB004573
4574 8240DB004574
4575 8240DB004575
4576 8230DB004576
4577 8230DB004577
4578 8230DB004578
4583 8220DB004583
4588 8350DB004588
4589 8350DB004589
4590 8350DB004590
4591 8350DB004591
4592 8220DB004592
4593 8220DB004593
4594 8220DB004594
4595 8220DB004595
4596 8220DB004596
4599 8230DB004599
4601 8230DB004601
4602 8230DB004602
4603 8230DB004603
4604 8230DB004604
4605 8230DB004605
4606 8230DB004606
4607 8230DB004607
4608 8230DB004608
4609 8230DB004609
4610 8230DB004610
4611 8230DB004611
4612 8230DB004612
4613 8230DB004613
4616 8230DB004616
4617 8230DB004617
4619 8230DB004619
4620 8230DB004620
4623 8230DB004623
4624 8230D

7166 8240DB007166
7167 8240DB007167
7168 8230DB007168
7169 8240DB007169
7170 8240DB007170
7172 8240DB007172
7173 8240DB007173
7174 8240DB007174
7175 8240DB007175
7176 8250DB007176
7177 8230DB007177
7178 8230DB007178
7180 8230DB007180
7181 8230DB007181
7183 8230DB007180
7184 8230DB007184
7185 8230DB007185
7186 8230DB007186
7187 8230DB007187
7188 8250DB007188
7189 8240DB007189
7191 8220DB007191
7192 8220DB007192
7193 8230DB007193
7194 8240DB007194
7195 8240DB007195
7196 8240DB007196
7197 8240DB007197
7199 8220DB007199
7201 8230DB007201
7204 8240DB007204
7207 8350DB007207
7208 8350DB007207
7214 8230DB007214
7216 8220DB007216
7217 8240DB007217
7218 8240DB007218
7219 8240DB007219
7220 8220DB007270
7222 8230DB007331
7224 8240DB007224
7226 8250DB007226
7227 8240DB007227
7229 8230DB007229
7230 8230DB007230
7231 8230DB007231
7233 8240DB007233
7234 8260DB007234
7236 8220DB007236
7237 8240DB007237
7238 8240DB007238
7239 8230DB007239
7240 8240DB001894
7241 8240DB007241
7242 8240DB007242
7243 8240D

In [28]:
# anomolous GTFS stops

for k, v in mapping.items():
    if len(v) > 12:
        print(k, v)

7347 8240DB007347_merged_7777
7400 8240DB007400_merged_7772
7401 8240DB007401_merged_7776
7675 gen:57102:7731:0:1
7676 gen:57102:7732:0:1
7677 gen:57102:7733:0:1
7690 gen:57102:7743:0:1
7701 gen:57102:7948:0:1
7703 gen:57102:7943:0:1


## Test if latter 4 digits of GTFS == DB code

In [29]:
#

padded_keys = []
codes = []

for k, v in mapping.items():
    pad = k.zfill(4)
    str(pad)
    padded_keys.append(pad)
    codes.append(v[-4:])


In [30]:
len(mapping)

4245

In [31]:
# print any mismatches between presumption that latter GTFS stop digits encoding == DB
# and the results of the haversein match.

number_diffs = 0

for i in range(len(padded_keys)):
    if padded_keys[i] != codes[i]:
        print(padded_keys[i], codes[i])
        number_diffs += 1
        
print("\n\n NUMBER OF DIFFERENCES", number_diffs)

0281 0278
0291 7591
0292 0288
0297 0298
0300 0299
0301 0298
0315 0313
0567 0707
0620 4380
0707 0567
0719 0696
0793 0792
1105 4392
1152 1143
1364 1383
1630 1631
1634 1628
1643 0207
1670 1691
1753 1726
1847 1695
2068 0435
2114 2115
2236 2215
2500 7216
3007 6326
3182 0450
3183 0449
3187 4984
3379 4669
3385 3413
3420 4669
3528 3499
3739 3726
4116 4093
4187 2964
4253 4267
4340 7181
4380 1184
4457 0448
4552 4555
4806 5155
4918 4923
5033 0253
5078 5075
5192 0336
6000 4533
6001 6002
6130 1267
6159 6160
6160 6164
6164 6159
6229 6230
6230 6229
6280 6285
6282 6283
6283 6282
6290 6289
6343 4758
6364 6366
7140 7139
7156 7163
7183 7180
7208 7207
7220 7270
7222 7331
7240 1894
7314 7313
7340 0751
7347 7777
7365 4591
7400 7772
7401 7776
7514 2278
7565 7088
7568 0299
7585 7586
7586 7585
7622 0315
7674 7738
7675 :0:1
7676 :0:1
7677 :0:1
7682 7738
7690 :0:1
7694 7771
7698 7697
7701 :0:1
7703 :0:1
7705 2720
7706 5140
7707 0499
7708 0516
7709 5140
7710 6251
7711 2720


 NUMBER OF DIFFERENCES 96


## Export dictionary

In [33]:
with open('/Users/davidodwyer/Documents/studyCS/Semester_3/data/derived_data/db_gtfs_stops_mapping.json', 'w') as json_file:
    json.dump(mapping, json_file, sort_keys=True)

# SEPARATE WORKFLOW: GENERATING STOP TO LAT/LON DICT

In [8]:
os.chdir('/Users/davidodwyer/Documents/studyCS/Semester_3/data/original_raw_data')

db_json = open('busstopinformation.json')
db_json_str = db_json.read()
db_data = json.loads(db_json_str)

In [9]:
db_stop_latlon = {}

In [10]:
for entry in db_data['results']:
    if entry['operators'][0]['name'] == 'bac':
        db_stop_latlon[entry['stopid']] = [float(entry['latitude']), float(entry['longitude'])]

In [11]:
db_stop_latlon

{'2': [53.35224111, -6.263695],
 '3': [53.35230694, -6.263783056],
 '4': [53.35256694, -6.264166111],
 '6': [53.35274389, -6.264443056],
 '7': [53.35283611, -6.264561944],
 '8': [53.35328111, -6.265155],
 '10': [53.35338694, -6.265383889],
 '11': [53.35709389, -6.264366944],
 '12': [53.35678889, -6.264623056],
 '14': [53.35853694, -6.262723889],
 '15': [53.36025111, -6.260973056],
 '16': [53.36641889, -6.305635],
 '17': [53.36306694, -6.258398056],
 '18': [53.36584889, -6.255931111],
 '19': [53.36713806, -6.255493056],
 '21': [53.37009194, -6.254291111],
 '22': [53.37350306, -6.252781944],
 '23': [53.37423194, -6.256245],
 '24': [53.375135, -6.260346111],
 '25': [53.37590306, -6.263611111],
 '27': [53.37943806, -6.266575],
 '28': [53.381575, -6.265988889],
 '29': [53.38348389, -6.265663056],
 '30': [53.37032111, -6.304578056],
 '31': [53.38603194, -6.268845],
 '32': [53.38619306, -6.27137],
 '33': [53.38875306, -6.271751111],
 '35': [53.38620806, -6.270908056],
 '36': [53.38612306, -6.

In [13]:
with open('/Users/davidodwyer/Documents/studyCS/Semester_3/data/derived_data/db_stop_latlon.json', 'w') as json_file:
    json.dump(db_stop_latlon, json_file, sort_keys=True)